In [1]:
import sys
from scipy.sparse.linalg import eigsh
import numpy as np
from scipy.spatial.distance import cosine
from scipy.sparse import csc_matrix, lil_matrix, coo_matrix, csr_matrix
from collections import Counter
import matplotlib.pyplot as plt 
from collections import OrderedDict
from itertools import chain
import operator
import pickle
%matplotlib inline


In [2]:
from tqdm import tqdm

### Вычисление симметричной матрицы слов $X$

Для анализа я взяла dump SimpleWiki и распарсила его с помощью парсера.

Всего я работаю с 84153 статьями.

In [3]:
#lines = ['i love tea', 'my friend', 'i am bee']
lines = [line.rstrip('\n') for line in open('parsed_simple_wiki.txt', encoding = 'utf-8')]
#lines = pickle.load(open('parsed_simple_wiki_cleaned_en.pkl','rb'))
print(len(lines))

84153


Текст после обработки выглядит вот так.

In [8]:
print(lines[0][:98])

april is the fourth month of the year and comes between march and may it is one of four months to 


Словарь уникальных слов имеет смысл предподсчитать один раз и сохранить, так как это достаточно долгий процесс.

In [9]:

word_set = set()
N = 0
for line in tqdm(lines):
    words = line.split(' ')
    N += len(words)
    word_set = word_set.union(set(words))
word_set = list(word_set)
print(len(word_set))


100%|████████████████████████████████████████████████████████████████████████████| 84153/84153 [45:18<00:00, 30.96it/s]

420401


In [10]:
#pickle.dump(word_set, open('word_set_small_wiki_en.pkl', 'wb'))

In [4]:
#import pickle
#word_set = pickle.load( open('word_set_small_wiki.pkl', 'rb'))

word_to_num = {}
num_to_word = {}
n = 0
for i, word in enumerate(word_set):
    word_to_num[word] = i
    num_to_word[i] = word
    n += 1
word_list = list(word_set)
del word_set


In [ ]:
word_to_num = {}
num_to_word = {}
n = 0
for i, word in enumerate(word_set):
    word_to_num[word] = i
    num_to_word[i] = word
    n += 1

На данный момент я работаю с пятиграммами (слово и два соседних). Я учитываю конец статьи, но пока не учитываю границы предложения. Т. к. во всех найденных мной скриптах знаки препинания пропадают при парсинге.

In [29]:
from time import time
start_ = time()
ss = [[word_to_num[w] for w in line.split(' ')] for line in lines]
print(round(time()-start_,2))
start_ = time()
s0 = [s for s in chain.from_iterable(ss)]
N_ = len(s0)
print(round(time()-start_,2))
start_ = time()
s1 = [s for s in chain.from_iterable([[s_cur[0]] + s_cur[:-1]  for s_cur in chain(ss)])]
print(round(time()-start_,2))
start_ = time()
s2 = [s for s in chain.from_iterable([[s_cur[0]] + [s_cur[0]] + s_cur[:-2]  for s_cur in chain(ss)])]
print(round(time()-start_,2))
start_ = time()
s1_ = [s for s in chain.from_iterable([[s_cur[0]] + [s_cur[0]] + s_cur[:-2] for s_cur in chain(ss)])]
print(round(time()-start_,2))
start_ = time()
s1_ = [s for s in chain.from_iterable([s_cur[1:] + [s_cur[-1]]  for s_cur in chain(ss)])]
print(round(time()-start_,2))
start_ = time()
s2_ = [s for s in chain.from_iterable([s_cur[2:] + [s_cur[-1]] + [s_cur[-1]]  for s_cur in chain(ss)])]
print(round(time()-start_,2))
start_ = time()

12.09
3.38
7.02
7.36
7.41
8.84
8.7


In [30]:
#word_list = list(word_set)
N_ = len(s0)
n, N_

(468473, 22592884)

In [9]:
#word_list = list(word_set)
N_ = len(s0)
n, N_

(468473, 22592884)

In [18]:
pickle.dump(s0, open('s0_end.pkl','wb'))
#pickle.dump(s1, open('s1_en.pkl','wb'))
#pickle.dump(s1_, open('s1__en.pkl','wb'))
#pickle.dump(s2, open('s2_en.pkl','wb'))
#pickle.dump(s2_, open('s2__en.pkl','wb'))
#s0 = pickle.load(open('s0_end.pkl','rb'))
#s1 = pickle.load(open('s1_end.pkl','rb'))
#s1_ = pickle.load(open('s1__end.pkl','rb'))
#s2 = pickle.load(open('s2_end.pkl','rb'))
#s2_ = pickle.load(open('s2__end.pkl','rb'))

In [31]:
%%time
# Подготавливаем индексы для матрицы (окрестность - правый и левый сосед)
s_len = list(range(len(s0)))
row = np.array(s1 + s1_+ s2 + s2_)
col = np.array(s_len + s_len + s_len + s_len)
data = np.ones(len(row))



Wall time: 34.9 s


In [32]:
%%time
#Задаем матрицу в coo формате.
A = coo_matrix((data, (row, col)), shape=(n, N_))
del data, row, col
#Переводим ее в csr формат, чтобы выполнять с ней необходимые операции.
A_csr = A.tocsr()
A_csr[A.nonzero()] = 1
#Вычисляем матрицу X
X = A_csr.dot(A_csr.transpose())
#print(A_csr.todense())
del A, A_csr, s1, s1_, s2, s2_, s_len

Wall time: 2min 10s


In [34]:
#pickle.dump(s0, open('s0_en.pkl','wb'))
pickle.dump(X, open('X_maxtrix5_end.pkl','wb'))
#s0 = pickle.load(open('s0.pkl','rb'))
#X = pickle.load(open('X_maxtrix5_end.pkl','rb'))

In [35]:
X.shape

(468473, 468473)

In [8]:
word_counts = Counter(s0)
stop_inds = [x[0] for x in word_counts.most_common(29)]
stop_words = [word_list[x] for x in stop_inds]

#['the', 'of', 'in', 'and', 'to', 'is', 'was', 'for', 'it', 'on', 'he', 'as', 'by', 'with', 'that', 'are', 'from', 'at', 'an', 'his', 'or', 'they', 'other', 'this', 'be', 'also', 'were', 'not', 'has']
print(stop_words)
vocab_inds = sorted([x[0] for x in word_counts.most_common(100029) if not x[0] in stop_inds])
vocab = [word_list[x] for x in vocab_inds]
word_freqs_short = [word_counts[x] for x in vocab_inds]
print(len(vocab))
#[6, 8, 17]
print(vocab_inds[:3])
word_to_num_short = {}
word_list_short = {}
for i, word in enumerate(vocab):
    word_to_num_short[word] = i
    word_list_short[i] = word
    
X_short = X[vocab_inds, :]
X_short = X_short[:, vocab_inds]
#(100000, 100000)
print(X_short.shape)

['the', 'of', 'in', 'and', 'to', 'is', 'was', 'for', 'it', 'on', 'he', 'as', 'by', 'with', 'that', 'are', 'from', 'at', 'an', 'his', 'or', 'they', 'other', 'this', 'be', 'also', 'were', 'not', 'has']
100000
[6, 8, 17]
(100000, 100000)
